# 4장 NB(Naive Bayes)


### 4.1 Classifying with Bayesian dicision theory

#### Pros
- Works with a small amount of data, handles multiple classes

#### Cons
- Sensitive to how the input data is prepared

#### Works with
- Nominal values


### 4.2 Conditional probability
- p(c|x) = p(x|c)p(c) / p(x)


### 4.3 Classifying with conditional probabilities

#### Bayesian decision theory
- If p1(x, y) > p2(x, y), then the class is 1
- If p2(x, y) > p1(x, y), then the class is 2

#### Bayesian classfication rule
- p(ci|x,y) = p(x,y|ci)p(ci) / p(x,y)
- If P(c1|x,y) > p(c2|x,y), the class is c1
- If P(c1|x,y) < p(c2|x,y), the class is c2


### 4.4 Document classification with naive Bayes

#### General approach to naive Bayes
1. Collect: Any method, We'll use RSS feeds in ths chapter.
2. Prepare: Numeric or Boolean values are needed.
3. Analyze: With many features, plotting features isn't helpful. Looking at histograms is a better idea.
4. Train: Calculate the conditional probabilities of the independent features.
5. Test: Calculate the error rate.
6. Use: One common application of naive Bayes is document classification. You can use naive Bayes in any classification setting. It doesn't have to be text.


### 4.5 Classifying text with Python
1. we're going to show how to transform lists of text into a vector of numbers
2. we'll show how to calculate conditional probabilities from these vectors
3. we'll create a classifier
4. we'll look at some practical considerations for implementing naive Bayes in Python

#### 4.5.1 Prepare: making word vectors from text

In [16]:
# loadDataSet Function
# output
#  - postingList: Traning data set
#  - classVec: Class labels
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', \
                  'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', \
                  'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', \
                  'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how',\
                  'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] #1 is abusive, 0 not
    return postingList,classVec

# createVocabList Function
# input
#  - dataSet: Training data set
# output
#  - list(vocabSet): unique vocablary set
def createVocabList(dataSet):
    vocabSet = set([])  # create an empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # create the union of two sets
    return list(vocabSet)

# setOfWords2Vec Function
# input
#  - vocabList: Training data Set
#  - inputSet: Input data
# output
#  - returnVec: Vector list of words
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList) # create a vector of all 0s
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print "the word: %s is not in my Vocabulary!" % word
    return returnVec

In [17]:
# loadDataSet example
listOPosts,listClasses = loadDataSet()
print "\nloadDataSet example:"
print listOPosts
print listClasses

# createVocabList example
myVocabList = createVocabList(listOPosts)
print "\ncreateVocabList example:"
print myVocabList

# setOfWords2Vec example
print "\nsetOfWords2Vec example:"
print setOfWords2Vec(myVocabList, listOPosts[0])
print setOfWords2Vec(myVocabList, listOPosts[3])




loadDataSet example:
[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], ['stop', 'posting', 'stupid', 'worthless', 'garbage'], ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
[0, 1, 0, 1, 0, 1]

createVocabList example:
['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']

setOfWords2Vec example:
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


#### 4.5.2 Train: calculating probabilities from word vectors

##### Pseudocode
- Count the number of documents in each class
- for every training document:
    - for each class:
        - if a token appears in the document ➞ increment the count for that token
        - increment the count for tokens
    - for each class:
        - for each token:
            - divide the token count by the total token count to get conditional probabilities
    - return conditional probabilities for each class

In [34]:
from numpy import *
# trainNB0 function
# input
#  - trainMatrix: Training data set
#  - trainCategory: Class labels
# output
#  - p0Vect: probability oflabel 0
#  - p1Vect: probability oflabel 1
#  - pAbusive: probability between labels
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # initialize probabilities
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = zeros(numWords); p1Num = zeros(numWords)
    p0Denom = 0.0; p1Denom = 0.0
    for i in range(numTrainDocs):
        # Vector addition
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # Element-wise division
    p1Vect = p1Num / p1Denom  # change to log()
    p0Vect = p0Num / p0Denom  # change to log()
    return p0Vect, p1Vect, pAbusive

In [37]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
print "trainMat:"
print trainMat

# trainNB0 example
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
print "\ntrainNB0 example:"
print p0V
print p1V
print pAb

trainMat:
[[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

trainNB0 example:
[ 0.04166667  0.04166667  0.04166667  0.          0.          0.04166667
  0.04166667  0.04166667  0.          0.04166667  0.04166667  0.04166667
  0.04166667  0.          0.          0.08333333  0.          0.
  0.04166667  0.          0.04166667  0.04166667  0.          0.04166667
  0.04166667  0.04166667  0.          0.04166667  0.          0.04166667
  0.04166667  0.125     ]

#### 4.5.3 Test: modifying the classifier for real-world conditions

In [40]:
from numpy import *
# trainNB0 function
# input
#  - trainMatrix: Training data set
#  - trainCategory: Class labels
# output
#  - p0Vect: probability oflabel 0
#  - p1Vect: probability oflabel 1
#  - pAbusive: probability between labels
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # initialize probabilities
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)
    p0Denom = 2.0; p1Denom = 2.0
    for i in range(numTrainDocs):
        # Vector addition
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # Element-wise division
    p1Vect = log(p1Num / p1Denom)  # change to log()
    p0Vect = log(p0Num / p0Denom)  # change to log()
    return p0Vect, p1Vect, pAbusive

In [41]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
print "trainMat:"
print trainMat

# trainNB0 example
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
print "\ntrainNB0 example:"
print p0V
print p1V
print pAb

trainMat:
[[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

trainNB0 example:
[-2.56494936 -2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -2.56494936
 -2.56494936 -3.25809654 -3.25809654 -2.15948425 -3.25809654 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -2.56494936 -3.25809654 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -1.8

In [46]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # Element-wise multiplication
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPost, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)

In [47]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


#### 4.5.4 Prepare: the bag-of-words document model

In [91]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [92]:
vec = bagOfWords2VecMN(myVocabList, listOPosts[0])
print vec

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]


### 4.6 Example: classifying spam email with naive Bayes

#### Example: using naive Bayes to classify email
1. Collect: Test files provided
2. Prepare: parse text into token vectors
3. Analyze: Inspect the tokens to make sure parsing was done correctly
4. Train: Use trainNB0() that we created earlier
5. Test: Use classifyNB() and create a new testing function to calculate hte error rate over a set of documents
6. Use: Build a complete program that will classify a group of documents and print misclassified documents to the screen

#### 4.6.1 Prepare: tokenizing text

In [57]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
listOfTokens = mySent.split()
print listOfTokens
listOfTokens = [tok.lower() for tok in listOfTokens if len(tok) > 0]
print listOfTokens

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M.L.', 'I', 'have', 'ever', 'laid', 'eyes', 'upon.']
['this', 'book', 'is', 'the', 'best', 'book', 'on', 'python', 'or', 'm.l.', 'i', 'have', 'ever', 'laid', 'eyes', 'upon.']


#### 4.6.2 Test: cross validation with naive Bayes

In [68]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():
    docList=[]; classList=[]; fullText=[]
    for i in range(1, 26):
        wordList = textParse(open('data/email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('data/email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = range(50); testSet = []
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print 'the error rate is: ', float(errorCount)/len(testSet)

In [69]:
spamTest()

the error rate is:  0.1


### 4.7 Example: using naive Bayes to reveal local attitudes from personal ads

#### Example: using naive Bayes to find locally used words
1. Collect: Collect from RSS feeds. We'll need to build an interface to the RSS feeds
2. Prepare: Parse text into token vectors
3. Analyze: Inspect the tokens to make sure parsing was done correctly
4. Train: Use trainNB0() that we created earlier
5. Test: We'll look at the error rate to make sure this is actually working. We can make modifications to the tokenizer to improve the error rate and results
6. Use: We'll build a complete program to wrap everything together. It will display the most common words given in two RSS feed

#### 4.7.1 Collect: importing RSS feeds

In [88]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1),reverse=True)
    return sortedFreq[:30]

def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList,fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet=[]
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != \
            classList[docIndex]:
            errorCount += 1
    print 'the error rate is: ',float(errorCount)/len(testSet)
    return vocabList,p0V,p1V

#### 4.7.2 Analyze: displaying locally used words

In [89]:
def getTopWords(ny, sf):
    import operator
    vocabList, p0V, p1V = localWords(ny, sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print "SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**"
    for item in sortedSF:
        print item[0]
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print "NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY **"
    for item in sortedNY:
        print item[0]

In [93]:
ny=feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
sf=feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
getTopWords(ny,sf)

the error rate is:  0.45
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
lady
very
enjoy
what
indian
ever
give
too
any
find
good
hang
does
all
whose
friendly
tired
love
use
sports
more
share
work
male
how
after
feels
age
seeking
doing
free
lol
open
marriage
bedroom
busy
unhappy
husband
attractive
chill
will
yes
happiness
lets
back
curious
single
evening
other
nice
pillows
oral
everything
brownie
minded
join
try
someday
drinks
giving
learned
water
great
makes
thats
ask
music
type
separated
started
company
downtown
town
divorced
movies
meet
excuses
foodie
high
something
thing
vibe
disappoint
advice
adventure
guys
man
harley
pleasure
over
still
fit
interesting
32ish
now
wont
friendship
sane
could
conversationalist
desi
soda
feel
spiritual
tomorrow
boring
tonight
livermore
ideas
useless
rides
hubby
face
points
wind
clean
26ish
going
international
him
ways
please
between
stories
reading
right
were
job
coffee
restaurant
sweet
period
wants
obese
life
educated
look
while
surprised
fun